In [1]:
import mysql.connector as connection
import pandas as pd
import numpy as np
import os
import sqlalchemy as sd
from tableone import TableOne
from openpyxl import load_workbook

In [2]:
last_edit = '11/20/2024'

In [3]:
pd.options.mode.chained_assignment = None  # default='warn'

In [4]:
Condition = "Diabetes"
USER = 'USER'
PWD = 'PWD'
HOST ='HOST'
DB = 'DB'
sheet_1 = 'sheet_1'

In [ ]:
v4_visits = r"C:\Users\breadsp2\Documents\Release_4.0.0\Participant_Visit_Info_4.0.0.xlsx"
v4_visits = pd.read_excel(v4_visits, sheet_name="Detailed_Report")
V4_fitler = False

In [5]:

#env = {}
#with open("test.env") as f:
#    for line in f:
#        (k, v) = line.split("=")
#        k = k.strip()
#        env[k] = v.strip()
        
#HOST = env["HOST"]
#DB = env["DB"]
#USER = env["USER"]
#PWD = env["PWD"]


In [6]:
creds = {'usr': USER, 'pwd': PWD, 'hst': HOST, "prt": 3306, 'dbn': DB}
connstr = "mysql+mysqlconnector://{usr}:{pwd}@{hst}:{prt}/{dbn}"
engine = sd.create_engine(connstr.format(**creds))
conn = engine.connect()

InterfaceError: (mysql.connector.errors.InterfaceError) 2003: Can't connect to MySQL server on '%-.100s:%u' (%s) (Warning: %u format: a real number is required, not str)
(Background on this error at: https://sqlalche.me/e/14/rvf5)

In [ ]:
mode='a'
if Condition in ["Diabetes", "Chronic_Lung_Disease","Viral_Infection"]:  #first terms in list, create the sheet
    mode = 'w'

In [ ]:
if V4_fitler == True:
    version_num = "4.2.0"
    output_folder = r"C:\Users\breadsp2\Desktop\Release_4.2.0"
else: 
    version_num = "5.2.0"
    output_folder = r"C:\Users\breadsp2\Desktop\Release_5.2.0"
file_sep = os.path.sep
release_date = '2024-10-01'

In [ ]:
cond_writer = pd.ExcelWriter(f'{output_folder}{file_sep}{sheet_1}_{version_num}.xlsx', mode=mode,  engine="openpyxl")

In [ ]:
norm_data = pd.read_sql(("Select Research_Participant_ID, Visit_Info_ID, SeroNet_Cohort, Normalized_Visit_Index from Normalized_Visit_Info"), conn)

if Condition == 'Cancer':
    cn_status = pd.read_sql((f"Select Visit_Info_ID, `Original Cancer Name` as 'Original_Names (Unique)', 'Not Reported' as 'ICD10 CODE',  " + 
                             "`Harmonized Cancer Name` as 'Harmonized_Value'  from Normalized_Cancer_Names"), conn)
    cn_status["Comorbidity"] = "Cancer"
 
else:
    cn_status = pd.read_sql((f"Select * from Normalized_Comorbidity_Visits"), conn)

In [ ]:
part_data = pd.read_sql(("SELECT Research_Participant_ID, Seronet_Participant_ID FROM Participant"), conn)
norm_data = norm_data.merge(part_data)                    

In [ ]:
cn_status = cn_status.query(f"Comorbidity == '{Condition}'") 

In [ ]:
diabetes_data = cn_status.merge(norm_data, how="outer")

In [ ]:
diabetes_data = diabetes_data.query("Normalized_Visit_Index == 1")

In [ ]:
if Condition == 'Cancer':
    uni_visit = list(set(diabetes_data["Visit_Info_ID"]))
    for curr_visit in uni_visit:
        try:
            test = diabetes_data.query(f"Visit_Info_ID == '{curr_visit}'")
            new_val = " | ".join(test["Harmonized_Value"])
            diabetes_data.loc[test.index, "Harmonized_Value"] = new_val
        except Exception as e:
            print(test)
    diabetes_data.drop_duplicates(inplace=True)

In [ ]:
demo_file = output_folder + file_sep + "Demographics_" + version_num + ".xlsx"
demo_data = pd.read_excel(demo_file, sheet_name="Detailed_Report")

In [ ]:
curr_visit_file = output_folder + file_sep + "Participant_Visit_Info_" + version_num + ".xlsx"
curr_visit_data = pd.read_excel(curr_visit_file, sheet_name="Detailed_Report")

In [ ]:
diabetes_data = diabetes_data.merge(demo_data["Seronet_Participant_ID"])
diabetes_data = diabetes_data.merge(curr_visit_data[["Seronet_Participant_ID", 'Normalized_Visit_Index']], how="right")

In [ ]:
col_list = ['Seronet_Participant_ID', 'SeroNet_Cohort', 'Normalized_Visit_Index', 'Comorbidity', 
            'Original_Names (Unique)', 'ICD10 CODE', 'Harmonized_Value']
diabetes_data  = diabetes_data[col_list]

In [ ]:
diabetes_data  = diabetes_data.query("Normalized_Visit_Index == 1")

In [ ]:
#diabetes_data.to_csv("diabetes_data.csv")

In [ ]:
#diabetes_data = diabetes_data.query("Normalized_Visit_Index == 1")
#diabetes_data.drop(["Comorbid_Name", "Orgional_Description_Or_ICD10_codes"], axis=1, inplace=True)
#diabetes_data.drop_duplicates(["Research_Participant_ID", "Visit_Info_ID", "Harmonized_Value"], inplace=True)

In [ ]:
if Condition == "Immunosuppressive_Condition":
    Condition = "Immunosuppressive"
    
if Condition == "Chronic_Neurological_Condition":
    Condition = "Chronic_Neurological"
    
if Condition == "Chronic_Oxygen_Requirement":
    Condition = "Chronic_Oxygen"
    

In [ ]:
diabetes_data.to_excel(cond_writer, Condition +'_Summary', index = False)

In [ ]:
cond_writer.close()